In [1]:
from azureml.core import Workspace

In [2]:
subscription_id = "d74ca76c-0758-4302-8894-12939776ca14" # The ID of the Azure Subscription
resource_group = "fastai-practise" # Name of a logical resource group
workspace_name = "Azure_projects" # The name of the workspace to look for or to create
workspace_region = 'northcentralus' # Location of the workspace
experiment_name = 'cars-classifier'
score_script = 'score_and_track.py'

In [3]:
ws = Workspace.create(name=workspace_name,
               subscription_id=subscription_id,
               resource_group=resource_group,
               create_resource_group=True,
               location=workspace_region,
               )

Deploying AppInsights with name azureproinsights03ddb98d.
Deployed AppInsights with name azureproinsights03ddb98d. Took 2.16 seconds.
Deploying KeyVault with name azureprokeyvault0276f48d.
Deploying StorageAccount with name azureprostoragecf2e28677.
Deployed KeyVault with name azureprokeyvault0276f48d. Took 17.47 seconds.
Deployed StorageAccount with name azureprostoragecf2e28677. Took 18.96 seconds.
Deploying Workspace with name Azure_projects.
Deployed Workspace with name Azure_projects. Took 18.2 seconds.


In [4]:
# ws = Workspace.get(name="Azure_projects", subscription_id='d74ca76c-0758-4302-8894-12939776ca14', resource_group='fastai-practise')

In [5]:
from azureml.core.model import Model

model = Model.register(model_path="export.pkl",
                          model_name="cars-classifier",
                          tags={'data': 'cars', 'method':'transfer learning','framework':'pytorch'},
                          description='Image classficiation service BMW vs Jagaur vs Mclauren',
                          workspace=ws)

Registering model cars-classifier


In [6]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig

myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'fastai==1.0.60','pillow==5.4.1',
                                               'torch', 'torchvision>=0.5.0'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score_and_track.py",
                                   environment=myenv)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
  - fastai==1.0.60
  - pillow==5.4.1
  - torch
  - torchvision>=0.5.0
channels:
- conda-forge



In [7]:
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'cars'

# Remove any existing service under the same name.
try:
   Webservice(ws, service_name).delete()
except WebserviceException:
   pass
    
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=4, 
                                               tags={'data': 'cars',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='Image classficiation service BMW vs Jagaur vs Mclauren')

In [8]:
service = Model.deploy(workspace=ws, 
                           name=service_name, 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

Running.............................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [9]:
service.get_logs()

'/bin/bash: /azureml-envs/azureml_1523582bfe50e56e88f1bbf0b060e680/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_1523582bfe50e56e88f1bbf0b060e680/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_1523582bfe50e56e88f1bbf0b060e680/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n2020-03-12T22:52:55,600960588+00:00 - gunicorn/run \n2020-03-12T22:52:55,602251294+00:00 - iot-server/run \n2020-03-12T22:52:55,602442995+00:00 - rsyslog/run \n2020-03-12T22:52:55,603471701+00:00 - nginx/run \n/bin/bash: /azureml-envs/azureml_1523582bfe50e56e88f1bbf0b060e680/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/usr/sbin/nginx: /azureml-envs/azureml_1523582bfe50e56e88f1bbf0b060e680/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_1523582bfe50e56e88

In [10]:
print(service.scoring_uri)

http://e411f54b-e480-44d9-ba36-5b29b0ace6fe.westus.azurecontainer.io/score


In [17]:
import json, requests
import os, base64
import urllib

#from PIL import Image

def preprocess(image):
    
    with open(image,mode='rb') as file:
        img = file.read()

    data = str(base64.b64encode(img), encoding='utf-8')  
    input_data = json.dumps({'data': data})
    return input_data


input_data = preprocess('test2.jpg')

result = service.run(input_data=input_data)
print(result)


{"category": "Jagaur", "confidence": 0.00086649798322469}
